#### Creating characters

- brainstorming
- deciding to write something into store inside the 'character object'

##### Loading env vars

In [ ]:
import os
from dotenv import load_dotenv

has_anything_loaded = load_dotenv()

if not has_anything_loaded:
    raise ValueError("No .env file found")

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

##### Story skeleton as a pydantic model

In [ ]:
from pydantic import BaseModel, Field
import uuid

class CharactersRelationTo(BaseModel):
    id: str
    self: str
    family: str
    relatives: str
    friends: str
    coworkers: str
    strangers: str
    authorityFigures: str
    locations: str
    
class CharactersCurrentStateOf(BaseModel):
    id: str
    physical: str
    mental: str
    spiritual: str
    
class CharactersIllness(BaseModel):
    id: str
    physical: str
    mental: str

class Character(BaseModel):
    id: str
    name: str
    age: int
    personality: str
    motivation: str
    pastChildhood: str
    pastTeenage: str
    pastYoungUpUntilNow: str
    strength: str
    weakness: str
    dailyRoutine: str
    phobias: str
    
    relationTo: CharactersRelationTo
    currentStateOf: CharactersCurrentStateOf
    illness: CharactersIllness
    
class Location(BaseModel):
    id: str
    name: str
    description: str
    
class Beat(BaseModel):
    id: str
    name: str
    actions: str = Field(description="The actions or events that take place which move the story forward")
    
class Scene(BaseModel):
    id: str
    name: str
    location: str
    dayOfTime: str
    beats: list[Beat]

class Sequence(BaseModel):
    id: str
    name: str
    scenes: list[Scene]    
    
class Act(BaseModel):
    id: str
    name: str
    summary: str
    sequences: list[Sequence]
    
class Episode(BaseModel):
    id: str
    theme: str
    acts: list[Beat]
    #TODO: fill out the rest of the fields

class Season(BaseModel):
    id: str
    theme: str
    episodes: list[Episode]
    #TODO: fill out the rest of the fields

#TODO: further descriptions and optimizations for the rest of pydantic classes

class Story(BaseModel):
    id: str = Field(default=str(uuid.uuid4()), description="unique identifier for the story")
    genre: str = Field(default="#toBeDefined", description="the genre of the story")
    logline: str = Field(default="#toBeDefined", description="the logline of the story")
    summary: str = Field(default="#toBeDefined", description="the summary of the story")
    plotPoints: list[str] = Field(default=[], description="a list of plot points that describe what happens throughout the entire story")
    character_ids: list[str] = Field(default=[], description="the characters' ids that move the story forward")
    season_ids: list[str] = Field(default=[], description="list of season ids")
    # characters: list[Character] = Field(description="the characters' ids that move the story forward")
    # seasons: list[Season] = Field(description="list of season ids")


##### sample story skeleton as a json string

In [2]:
# ──────────────────────────────────────────────────────────────────────── #
#  Example payload that satisfies *all* Pydantic models with *empty* data
# ──────────────────────────────────────────────────────────────────────── #
story_json = {
    # ─────── Story ────── #
    "id": "123",
    "genre": "",
    "logline": "",
    "summary": "",
    "plotPoints": [],

    # ─────── Characters ────── #
    "characters": [
        {
            "id": "",
            "name": "",
            "age": 0,
            "personality": "",
            "motivation": "",
            "pastChildhood": "",
            "pastTeenage": "",
            "pastYoungUpUntilNow": "",
            "strength": "",
            "weakness": "",
            "dailyRoutine": "",
            "phobias": "",

            "relationTo": {
                "id": "",
                "self": "",
                "family": "",
                "relatives": "",
                "friends": "",
                "coworkers": "",
                "strangers": "",
                "authorityFigures": "",
                "locations": ""
            },

            "currentStateOf": {
                "id": "",
                "physical": "",
                "mental": "",
                "spiritual": ""
            },

            "illness": {
                "id": "",
                "physical": "",
                "mental": ""
            }
        }
    ],

    # # ─────── Locations (referenced by scenes) ────── #
    # "locations": [
    #     {"id": "", "name": "", "description": ""}
    # ],

    # ─────── Seasons ────── #
    "seasons": [
        {
            "id": "",
            "theme": "",
            "episodes": [
                {
                    "id": "",
                    "theme": "",

                    # Episode → acts (type `list[Act]`)
                    "acts": [
                        {
                            "id": "123",
                            "name": "",
                            "summary": "",

                            # Act → sequences (empty list – “empty act”)
                            "sequences": [
                                # One empty Sequence (no scenes → “empty sequence”)
                                {
                                    "id": "",
                                    "name": "",
                                    "scenes": [
                                        # One empty Scene (no beats → “empty scene”)
                                        {
                                            "id": "",
                                            "name": "",
                                            "location": "",
                                            "dayOfTime": "",
                                            # Beat list left empty (no actions)
                                            "beats": [
                                                {
                                                    "id": "",
                                                    "name": "",
                                                    "actions": ""
                                                }
                                            ]
                                        }
                                    ]
                                }
                            ]
                        }
                    ]
                }
            ]
        }
    ]
}
# ──────────────────────────────────────────────────────────────────────── #

# ──────────────────────────────────────────────────────────────────────── #
#  How to validate it against your models
# ──────────────────────────────────────────────────────────────────────── #
# from your_models import Story        # ← the file where you defined the classes
# story = Story.parse_obj(story_json)   # → no validation errors


# story = Story.model_validate(story_json, by_alias=True)
# story = Story(**story_json)
# story_json

##### Tools for persistance

In [68]:
from typing import Any
from langchain.tools import tool, ToolRuntime
from pydantic import ValidationError
from typing import Union, Literal

stories_in_memory_db: dict[str, Story] = {}

@tool
def read_or_create_story(story_id: Union[str, None], runtime: ToolRuntime) -> Story:
    """
    Searches for a story with the given story_id. If no such story is found, it creates a new story and returns it.
    
    Args:
        story_id: the id of the story that wants to be read. This is an optional parameter. It is okay to pass nothing and it will create a new story.
    
    Return: story object
    """
    writer = runtime.stream_writer
    writer("read story is called")
    
    def create_new_story():
        writer("creating a new story")
        new_story = Story()
        stories_in_memory_db[new_story.id] = new_story
        return new_story
        
    if story_id is None:
        return create_new_story()    
    
    writer("checking the in-memory db for the story with id {}".format(story_id))
    result = stories_in_memory_db.get(story_id)
    
    if result is None:
        return create_new_story()
    else:
        return result

@tool
def update_story(updated_story: str, runtime: ToolRuntime) -> Union[Literal["failure"], Literal["success"]]:
    """
    Overwrites the existing outdated story with the given updated_story
    
    Args:
        updated_story: contains the new version of story as a json string
    """
    
    writer = runtime.stream_writer
    
    writer("update_story is called")
    print("update_story is called")
    try:
        print("validation begins")
        validated_story = Story.model_validate_json(updated_story)
        print("validation succeeded")
        stories_in_memory_db[validated_story.id] = validated_story
        print("updated the story in-memory")
        print("returning")
        return "success"
    
    except ValidationError as ve:
        print("Validation of the story str failed")
        print(ve.errors())
        print("returning")
        return "failure"
    

@tool
def write_to_database(updated_story: Story):
    """write the new updates so that they are persisted"""
    pass

@tool
def read_from_database():
    """read the latest state"""
    pass


# @tool
# def update_story(updated_story: dict[str, Any]):
#     """
#     Update the old version of the story if there is a new version
    
#     updated_story: this contains the updated version of the story which will replace the old version
    
#     """
#     print("update_story: called")
#     print("update_story: updated_story = {}".format(updated_story))
#     story_json = updated_story

    


##### Agent definition

In [74]:
from langchain_ollama.chat_models import ChatOllama
from langchain.agents import create_agent
from langchain.chat_models import init_chat_model

my_agent = ChatOllama(
    model="gpt-oss:20b", 
    reasoning="high",
    verbose=True,
    
    # temperature=0.5,
    # disable_streaming=True
    # format=Story.model_dump_json(), # type: ignore
).bind_tools([read_or_create_story, update_story])

# my_agent = my_agent.bind_tools([update_story_variable])

# my_agent = create_agent("ollama:gpt-oss:20b", tools=[update_story_variable], response_format=Story)
# my_agent = init_chat_model("ollama:gpt-oss:20b",)


# my_agent = init_chat_model("openai:gpt-5.2") # testing cloud gpt 5.2
# my_agent.bind_tools([update_story_variable]) # binding for the (local) agent


##### Running the agent

In [75]:
from langchain.messages import SystemMessage, HumanMessage, AIMessage, AIMessageChunk
from IPython.display import Markdown, display
from typing import Union

# def chatWithScenarist(newSession: bool = True, past_messages: list[Union[SystemMessage, HumanMessage, AIMessage]] = []):

messages: list[Union[SystemMessage, HumanMessage, AIMessage]] = [
        SystemMessage(
            """
            You are an expert screenwriter. You are here to provide ideas and inspirations.
            Do not update something before I confirm it.
            Do not make up ids.
            Tell what you did, even if you only updated something.
            Always go through the story json fields one by one and do not try to answer more than one field at a time. 
            Provide new ideas and critics to the user's ideas and decisions. Base those ideas and critics on past scenarios, movies and even maybe other art forms.
            Do not sugercoat anything or do not try to make it look pleasing.
            
            """.format(story_json)),
        #HumanMessage("I want to write a sitcom about two naive guys in Berlin and their struggles regarding love life")
    ]

            # Do not stop simply after calling a tool; tell me what you did and move forward.
            
            # Return complete story json when you update something.

            # Use your intelligence to provide a deep and or explorative answer for the given field.
            # Make your answers only so long as it is necessary.
            # You can be humorous and suggestive and creative. Remember that you are an artist and you don't have to be correct all the time. Be explorative.

#response = my_agent.invoke(messages)

# 5️⃣  Run in a loop (or UI)
while True:
    user_input = input("You: ")
    if user_input.lower() in {"exit", "quit"}:
        break
    
    print("user: {}".format(user_input))
    
    messages.append(HumanMessage(user_input))
    
    response = my_agent.invoke(messages)
    print(response)
    # ---
    # response_text: Union[str, None] = None
    # streaming_counter = 0
    # for chunk in my_agent.stream(input=messages):
    # # for chunk in my_agent.stream('[{"role": "user", "content": "sdfsdf"}]'):
    #     # latest_message = chunk["messages"][-1]
    #     latest_message = chunk
    #     # print("text: {} \ntype: {} \ncontent: {} \ncontent blocks: {} \nresponse metadata: {} \nat loop: {}".format(chunk.text, chunk.type, chunk.content, chunk.content_blocks, chunk.response_metadata, streaming_counter))
    #     print(chunk.content, end="", flush=True)
        
    #     if chunk.content and chunk.content_blocks[0]["type"] == "text":
    #         if response_text is None:
    #             response_text = chunk.content # type: ignore
    #         else:
    #             response_text += chunk.content # type: ignore
                
    #     streaming_counter += 1

    # response = AIMessage(content=response_text)
    
    messages.append(response)
    
    display(Markdown(response.content))
    
# chatWithScenarist(False, messages) # type: ignore

user: I want to write a story
content='' additional_kwargs={'reasoning_content': 'The user says "I want to write a story". According to the instructions: We are an expert screenwriter providing ideas and inspirations. We should not update something before the user confirms it. We should not make up ids. We need to go through the story json fields one by one and do not try to answer more than one field at a time. We should provide new ideas and critics to the user\'s ideas and decisions. We should not sugarcoat anything or try to make it look pleasing.\n\nWe need to ask the user what story fields we should address. According to the system instruction, we have to read or create a story. But the user hasn\'t provided any story id or fields. The user just says "I want to write a story".\n\nThus the best approach is to start by creating a new story. The instructions say we can call the function read_or_create_story with no story_id. But we should ask user if they want to create a new story?

user: what is next? 
content='' additional_kwargs={'reasoning_content': 'We need to respond as an expert screenwriter, following instructions: "Always go through the story json fields one by one and do not try to answer more than one field at a time." We have a story object, probably returned from read_or_create_story. We don\'t see it yet; we must fetch it. We need to call the function read_or_create_story with no story_id to get a new story. That is the first step.\n\nBut the user asked "what is next?" We need to respond with instructions on next steps: e.g., "We need to populate the \'genre\' field." But we must do it field by field. The instruction: "Always go through the story json fields one by one and do not try to answer more than one field at a time." That means we can\'t give multiple field suggestions at once. We should ask what the user wants to fill next. Possibly "What genre would you like?" etc.\n\nWe need to tell them that we have the story object. We haven\'t gotten it